<a href="https://colab.research.google.com/github/ishaan1234/NLP-Practice/blob/main/IMDB_Movie_Review_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !unzip "/content/drive/MyDrive/IMDB Project/IMDB Movie.zip" -d "/content/drive/MyDrive/IMDB Project"

Archive:  /content/drive/MyDrive/IMDB Project/IMDB Movie.zip
  inflating: /content/drive/MyDrive/IMDB Project/IMDB Dataset.csv  


In [ ]:
import pandas as pd
import spacy 
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [ ]:
movies=pd.read_csv(f"/content/drive/MyDrive/IMDB Project/IMDB Dataset.csv")
movies.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
from bs4 import BeautifulSoup

nlp=spacy.load('en_core_web_sm')

def remove_html(review):
  soup=BeautifulSoup(review, "html.parser")
  new_text=soup.get_text(separator=" ")
  return new_text.lower()

movies["review"]=movies["review"].apply(remove_html)

<ipython-input-18-759f8fb26af4>:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup=BeautifulSoup(review, "html.parser")


In [ ]:
def remove_punct(text):
  punct="[!@#$%^&*/\|.?<>:;_]"
  clean_text=re.sub(punct, '', text)
  return clean_text

movies['review']=movies['review'].apply(remove_punct)

stop = stopwords.words('english')
movies['review'] = movies['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
stemming=PorterStemmer()
def stem_text(text):
  tokens=nltk.word_tokenize(text)
  stemmed_words = [stemming.stem(token) for token in tokens if token not in stop]
  stemmed_text= " ".join(stemmed_words)
  return stemmed_text

movies['review'] = movies['review'].apply(stem_text)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(movies['review'])


In [ ]:
movies['sentiment']=movies['sentiment'].map({'negative':0, 'positive':1})

In [ ]:
movies.sentiment.value_counts()

1    25000
0    25000
Name: sentiment, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, movies['sentiment'], test_size=0.2)


In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg=LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)
log_reg.score(X_test, y_test)

0.8804